In [101]:
class Node():
    """A node in the suffix tree. 
    
    value
        As a identifier, custom for each node except leav nodes
    """
    def __init__(self,value = "$"):
        self.part_of_string = []
        self.edges = []
        self.identifier = value
    
    def set_part_of_string(self,num):
        """
        Defines which substring is "part" of the node
        """
        self.part_of_string.append(num)
        
    def set_edges(self,edge):
        """
        the edge set of the node
        """
        self.edges.append(edge)
    
    def __repr__(self):
        return "Node : {}\n  Part of string: {} \n  Children: {}".format(self.identifier,self.part_of_string,self.edges)

In [108]:
node1 = Node(1)
node1.set_part_of_string(1)
node1.set_edges(edge)
print(node1)

node2 = Node(2)
node2.set_part_of_string(3)
node2.set_edges(edge2)
print(node2)

Node : 1
  Part of string: [1] 
  Children: [Value: A, From Node: 1, To Node: 2]
Node : 2
  Part of string: [3] 
  Children: [Value: B, From Node: 2, To Node: $]


In [187]:
class Edge():
    """
    An Edge in the suffix tree
    
    value
        The value/label of the edge
    
    parrent_node
        The node at which the edge starts
        
    child_node
        The Node where the edge is heading to /ending
    """
    
    def __init__(self,value,parrent_node,child_node=Node()):
        self.value= value
        self.from_node = parrent_node
        self.to_node = child_node
        
    def set_to_node(self,node):
        """
        Sets the Node to which the edge is heading
        """
        self.to_node = node
        
    def __repr__(self):
        return "Value: {}, From Node: {}, To Node: {}\n".format(self.value,self.from_node.identifier,self.to_node.identifier)  


In [188]:
edge = Edge("A",node1,node2)
edge2 = Edge("B",node2)
print(edge)
print(edge2)

Value: A, From Node: 1, To Node: 2

Value: B, From Node: 2, To Node: $



In [709]:
class GenralizedSuffixTree():
    """
    A naiv implementation of the generalized suffix tree 
    """
    def __init__(self,sequences):
        """
        sequences
            A list of strings which should be added to the GST
            
        identifier
            Starting identifier to label the nodes
            
        root
            root node of the GST
        """
        self.sequences = sequences
        self.node_identifier = 0
        self.root = Node(0)
    
    
    def add_suffix(self,suffix,part_of_string):
        """
            Adds Suffix to the existing SuffixTree
            
            suffix:
                a suffix of a sequence
            
            part_of_string:
                number indicating to which sequence the current suffix belongs
        """
        suffix = suffix
        # root node as current node
        current_node = self.root
        # for every character in the suffix
        while suffix:
            char=suffix[0]
            #create list of edges from node
            edges = [edge.value for edge in current_node.edges]
            # Case 1: char is not an edge of the current node
            if char not in edges:
                # increase node_identifier for next node
                self.node_identifier += 1
                
                # if the next node is not a end node (because there is only one char left in the suffix)
                if len(suffix) > 1:
                    #create new Node with label
                    child_node = Node(self.node_identifier)
                else:
                    #create new node with end label $
                    child_node = Node()
                
                # the new node is a part of the current suffix
                if len(suffix) <= 1:
                    child_node.set_part_of_string(part_of_string)
                
                #Here we create a new edge with the current char as value, from the current node to the child node
                current_node.set_edges(Edge(char,current_node,child_node))
                
                # now we change our current node to the new created node
                current_node= child_node
                # and shorten the suffix by the character we just added into the tree
                suffix = suffix[1:]
            
            # Case 2: char is already an edge of the current node
            else:
                
                while True:
                    if len(suffix) <= 1:
                        current_node.set_part_of_string(part_of_string)
                    #find the edge that corresponds to the char of our suffix
                    for edge in current_node.edges:
                        if char == edge.value:
                            #we set our current node to the "to_node" of the edge in which we found our char
                            current_node = edge.to_node
                            #the current node is a part of the current suffix
                            #  shorten the suffix by the character we already have in the tree
                            suffix = suffix[1:]
                            break
                    break
                        
    
    def construct_tree(self):
        """
        Constructs the tree given the input Sequences using the add function
        """
        for part_of_string, seq in enumerate(self.sequences):
            for suffix in self.suffix(seq):
                self.add_suffix(suffix,part_of_string)
        
        return self
    
    
    def prefix(self,string):
        """
        Given a string, yield all the prefixes of the string starting by the whole string
        """
        for i in range(len(string)):
            string_to_yield = string
            string = string[:-1]
            yield string_to_yield
            
    def suffix(self,string):
        """
        Given a string, yield all the suffixes of the string starting with the whole string
        """
        end = len(string)
        for i in range(end):
            yield string[i:end]
    
    def search_all(self, pattern):
        for prefix in self.prefix(pattern):
            print(self.search(prefix))
    
    def search(self, pattern):
        """
        Given a pattern, look if any prefix of the pattern is in the GST
        
        pattern
            the pattern we want to search for
        """
        for prefix in self.prefix(pattern):
            print("\nLooking for {}\n".format(prefix))
            #set current node as root_node
            current_node = self.root
            
            if not current_node.edges:
                return False , []
            for char in prefix:
                char_not_found = True
                
                for edge in current_node.edges:
                    if char == edge.value:
                        char_not_found = False
                        current_node = edge.to_node
                        break
                
                if char_not_found:
                    return False , [] 
            if current_node.identifier == "$":
                return True, current_node.part_of_string
            else:
                return False, []
                

In [710]:
seqs = ["ACCB","ABCC","ABBB"]
tree = GenralizedSuffixTree(seqs).construct_tree()

In [712]:
tree.search_all("CC")


Looking for CC

(False, [])

Looking for C

(False, [])


In [707]:
tree.root.edges

[Value: A, From Node: 0, To Node: 1,
 Value: C, From Node: 0, To Node: 5,
 Value: B, From Node: 0, To Node: 9]